In [1]:
import sys
sys.path.append('..')
from src.utilities import *

import warnings
warnings.filterwarnings("ignore")

import logging, sys
logging.disable(sys.maxsize)

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

## 1. Prophet feature extraction

### 1.1 Extract predictions for each month-brand-region using previous information

In [ ]:
sales_train = pd.read_csv(os.path.join(raw_path, 'sales_train.csv'))

def extract_prophet_features(raw_sales: pd.DataFrame) -> pd.DataFrame:
    '''Provides prediction, additive term and trend term for every month-region-brand'''
    
    months_to_predict = ['2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01',
                         '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08']

    brands_to_predict = ['brand_12_market', 'brand_3', 'brand_3_market']

    results_prophet = pd.DataFrame()

    for reg in tqdm(raw_sales.region.unique()):
        for brand in brands_to_predict:
            for mon in months_to_predict:

                current = raw_sales[(raw_sales.brand == brand) & 
                                      (raw_sales.region == reg) & 
                                      (raw_sales.month < mon)][['month', 'sales']]
                current.columns = ['ds', 'y']
                current.ds = pd.to_datetime(current.ds)

                m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)

                m.fit(current, iter = 60)

                prediction = m.predict(pd.DataFrame({'ds': [mon]}))

                to_append = pd.DataFrame({'month': [mon],
                                          'brand': [brand],
                                          'region': [reg],
                                          'trend': [prediction.trend[0]],
                                          'prediction': [prediction.yhat[0]],
                                          'additive_terms': [prediction.additive_terms[0]]
                                         })

                results_prophet = pd.concat([results_prophet, to_append])
                
    trend_features = results_prophet.pivot(index = ['month', 'region'], columns = 'brand', values = 'trend').reset_index()
    trend_features.columns = ['month', 'region', 'prophet_b12_market_trend', 'prophet_b3_trend', 'prophet_b3_market_trend']

    yhat_features = results_prophet.pivot(index = ['month', 'region'], columns = 'brand', values = 'prediction').reset_index()
    yhat_features.columns = ['month', 'region', 'prophet_b12_market_yhat', 'prophet_b3_yhat', 'prophet_b3_market_yhat']

    additive_features = results_prophet.pivot(index = ['month', 'region'], columns = 'brand', 
                                              values = 'additive_terms').reset_index()
    additive_features.columns = ['month', 'region', 'prophet_b12_market_add', 'prophet_b3_add', 'prophet_b3_market_add']
    
    prophet_features = trend_features.merge(yhat_features, how = 'left', on = ['month', 'region'])
    prophet_features = prophet_features.merge(additive_features, how = 'left', on = ['month', 'region'])
    prophet_features.reset_index(drop = True, inplace = True)
    
    return prophet_features

prophet_features = extract_prophet_features(sales_train)
prophet_features.head()

 14%|██████████▊                                                                   | 28/201 [57:53<7:13:44, 150.43s/it]

### 1.2 Store prophet features

In [ ]:
prophet_features.to_csv(os.path.join(interim_path, 'prophet_features.csv'), index = False)